In [1]:
from dotenv import load_dotenv

load_dotenv() ##load all the nevironment variables
import os
import google.generativeai as genai

from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.discovery import build
import pandas as pd
# from youtube_api import YoutubeDataApi

d:\Anaconda\envs\abit_mod1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
prompt = """You are Yotube video summarizer. You will be taking the transcript text
and summarizing the entire video and providing the important summary in points
within 250 words. Please provide the summary of the text given here:  """

In [63]:
def extract_transcript_details(video_id):
    try:
        transcript_text = YouTubeTranscriptApi.get_transcript(video_id)

        transcript = ""
        for i in transcript_text:
            transcript += " " + i["text"]

        return transcript

    except Exception as e:
        raise e

In [64]:
## getting the summary based on Prompt from Google Gemini Pro
def generate_gemini_content(transcript_text,prompt):

    model = genai.GenerativeModel("gemini-pro")
    response = model.generate_content(prompt + transcript_text)
    return response.text

In [4]:
youtube = build('youtube', 'v3', developerKey=os.getenv("YOUTUBE_API_KEY"))


In [37]:
# channel_name = 'Channel Name'
def get_channel_id(channel_name):
    youtube = build('youtube', 'v3', developerKey=os.getenv("YOUTUBE_API_KEY"))

    channel_request = youtube.channels().list(
        part='id',
        forUsername=channel_name,
        maxResults=5
    )
    channel_response = channel_request.execute()

    # Get the channel ID
    channel_id = channel_response['items'][0]['id']

    return channel_response



In [38]:
channel_response = get_channel_id('krishnaik06')

In [40]:
channel_response

{'kind': 'youtube#channelListResponse',
 'etag': '1JNpVGLiUp94-zv5eFSAvMCOQDc',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 't3bRHzQgRuxznY6aGFQEtKLpDFU',
   'id': 'UCQzSk4RM5JQ7UJJK5CWjPsQ'}]}

In [19]:
channel_id=get_channel_id('MostlySane')

In [20]:
channel_id

'UCQzSk4RM5JQ7UJJK5CWjPsQ'

In [ ]:
UCvCyIiKSCA1fHKSCOKJyjXA

In [43]:
def get_top_5_recent_videos(channel_id):
    youtube = build('youtube', 'v3', developerKey=os.getenv("YOUTUBE_API_KEY"))

    request = youtube.search().list(
        
        part="snippet",
        channelId=channel_id,
        order="date",               #  'searchSortUnspecified', 'date', 'rating', 'viewCount', 'relevance', 'title', 'videoCount'
        type="video",
        maxResults=5
    )

    response = request.execute()

    video_ids = [item['id']['videoId'] for item in response['items']]

    video_titles = [item['snippet']['title'] for item in response['items']]


    # return video_ids,video_titles
    return response

In [46]:
response = get_top_5_recent_videos('UCvCyIiKSCA1fHKSCOKJyjXA')

In [47]:
response

{'kind': 'youtube#searchListResponse',
 'etag': '74rA4V-cZEovQkJnJgqIUlmjsok',
 'nextPageToken': 'CAUQAA',
 'regionCode': 'IN',
 'pageInfo': {'totalResults': 195061, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#searchResult',
   'etag': 'v3_sY8jCo1rtt2p1Jz3EcESdvHc',
   'id': {'kind': 'youtube#video', 'videoId': 'hubXjZr31AQ'},
   'snippet': {'publishedAt': '2024-04-05T06:02:42Z',
    'channelId': 'UCvCyIiKSCA1fHKSCOKJyjXA',
    'title': 'It’s working.',
    'description': 'Click here to Subscribe :- http://bit.ly/PrajaktaKoli Click here to buy my merch - https://mostlysane.themerchbay.com/ Link To My ...',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/hubXjZr31AQ/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/hubXjZr31AQ/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/hubXjZr31AQ/hqdefault.jpg',
      'width': 480,
      'height': 360}},
    'channelTitle

In [27]:
video_ids,video_titles = get_top_5_recent_videos(channel_id)

In [28]:
video_titles

['Testmovie2.avi',
 '7: Final part 1',
 '8:Final Part two',
 '6: Josh/Goat',
 '5: The Commons']

In [116]:
transcript0=extract_transcript_details(video_ids[0])

In [114]:
transcript1 = extract_transcript_details(video_ids[1])

In [115]:
transcript4 = extract_transcript_details(video_ids[4])

In [137]:
len(transcript4*24+transcript1+transcript0)

152032

In [141]:
152032/12888

11.796399751707014

In [136]:
summary=generate_gemini_content(transcript4*24+transcript1+transcript0,prompt)

In [140]:
summary

"**Summary: Python Roadmap for Aspiring Experts**\n\n**Point 1: Determine Your Goal and Timeline**\n\n* Define your learning purpose (e.g., data science, web development)\n* Set a realistic timeline (e.g., 3 months for data science)\n\n**Part 1: Python Syntax and Basic Operations**\n\n* Jupyter Notebook\n* Basic arithmetic operations\n* Variables, control and conditional structures (if/else)\n* Loopings (for/while)\n* User input, strings, integers, floats, type casting (2 days)\n\n**Part 2: Built-in Data Structures and Functions**\n\n* Python strings, lists, booleans, sets, dictionaries, tuples (3 days)\n* Functions in Python: user-defined and built-in\n* Exception handling\n* Importing libraries\n\n**Part 3: Object-Oriented Programming (OOP)**\n\n* Classes, objects, methods\n* Inheritance, polymorphism, encapsulation (4 days)\n\n**Part 4: Libraries for Python**\n\n* NumPy, Pandas, Matplotlib, Seaborn, Scikit-learn\n* Web development frameworks: Flask, Django (4 days)\n\n**Intermediate

In [104]:
transcript[:5000]

" hello all my name is krishnaik and welcome to my YouTube channel so guys finally I'm going to start the seven days end-to-end machine learning project implementation with mlops and cicd pipelines I had announced this last month that I'm planning to do it and I had asked question that whether you want recorded videos or live session and many of you had actually demanded for the recorded videos so yes finally from tomorrow that is Sunday that is 5th March uh at 8 pm you will be getting the record"

In [95]:
for video in video_ids:

    transcript = extract_transcript_details(video)
    print(len(transcript))

9306
30118
279708
360054
4692


In [76]:
# get_channel_id_from_user

In [68]:
# list(response.keys())

In [78]:
[item['snippet']['title'] for item in response['items']]


['Perfect Roadmap To Become AI Engineers In 2024 With Free Videos And Materials',
 'It Is Easy To Start But Difficult to ...',
 'Groq-LPU™ Inference Engine Better Than OpenAI Chatgpt And Nvidia',
 'AI vs ML vs DL vs Generative Ai',
 '3-Langchain Series-Production Grade Deployment LLM As API With Langchain And FastAPI']

In [77]:
# response['items']

[{'kind': 'youtube#searchResult',
  'etag': 'I1FIWX14CjJszoUSorHaC3xo71o',
  'id': {'kind': 'youtube#video', 'videoId': 'bkhUjwJbP1k'},
  'snippet': {'publishedAt': '2024-04-05T13:53:47Z',
   'channelId': 'UCNU_lfiiWBdtULKOw6X0Dig',
   'title': 'Perfect Roadmap To Become AI Engineers In 2024 With Free Videos And Materials',
   'description': 'Artificial intelligence (AI) is a branch of computer science that involves programming machines to think like human brains.',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/bkhUjwJbP1k/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/bkhUjwJbP1k/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/bkhUjwJbP1k/hqdefault.jpg',
     'width': 480,
     'height': 360}},
   'channelTitle': 'Krish Naik',
   'liveBroadcastContent': 'none',
   'publishTime': '2024-04-05T13:53:47Z'}},
 {'kind': 'youtube#searchResult',
  'etag': 'cMucgA-o57gc61WHrIN

In [70]:
for key in list(response.keys()):
    print(response[key])

youtube#searchListResponse
r4yc6N3E5muh1TYC5ZVO3NCaWS0
CAUQAA
IN
{'totalResults': 151217, 'resultsPerPage': 5}
[{'kind': 'youtube#searchResult', 'etag': 'I1FIWX14CjJszoUSorHaC3xo71o', 'id': {'kind': 'youtube#video', 'videoId': 'bkhUjwJbP1k'}, 'snippet': {'publishedAt': '2024-04-05T13:53:47Z', 'channelId': 'UCNU_lfiiWBdtULKOw6X0Dig', 'title': 'Perfect Roadmap To Become AI Engineers In 2024 With Free Videos And Materials', 'description': 'Artificial intelligence (AI) is a branch of computer science that involves programming machines to think like human brains.', 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/bkhUjwJbP1k/default.jpg', 'width': 120, 'height': 90}, 'medium': {'url': 'https://i.ytimg.com/vi/bkhUjwJbP1k/mqdefault.jpg', 'width': 320, 'height': 180}, 'high': {'url': 'https://i.ytimg.com/vi/bkhUjwJbP1k/hqdefault.jpg', 'width': 480, 'height': 360}}, 'channelTitle': 'Krish Naik', 'liveBroadcastContent': 'none', 'publishTime': '2024-04-05T13:53:47Z'}}, {'kind': 'youtube#

In [71]:
video_ids

['bkhUjwJbP1k', 'WzV1eUiGv2M', 'rHphpyf0i0I', 'X7Zd4VyUgL0', 'XWB5DXP-DO8']

 hello all my name is krishak and welcome to my YouTube channel so guys many many people were requesting quickly come up with the AI engineer road map um you know probably in every video from the day that I have actually uploaded a video about what does an AI engineer do you know and in the comments many people were saying CR please go ahead and upload this AI engineer road map complete AI engineer road map and uh yes I it took me some amount of time because uh I always provide you road map just not by saying you have to learn this or that I provide road map by providing you videos free videos complete course free videos along with that documentation link if I consider with respect to materials end to endend projects everything is basically included in all this kind of road maps whatever road maps that you probably see Let It Be ml Engineers data science data analyst I always make sure to do that you know now again uh in this specific road map you'll be completely seeing the AI Enginee

In [ ]:
'''   
Task in this module

1. Built a function to find channel_id from youtube channel names.  done
2. Select filter method by which we want to select videos from competitors youtube channel (right now its picking latest 5 videos). done 
3. Compare and validate the results from filtered results. done

4. Find accurate model or algorithm to summarize any length of transcription (divide transcriptions into chunks and feed the chunks
                                                                             to the model with some overlap between them for context)

Will take langchain framework for recursive text splitting and use ether open api key or google gemini model to summarize the transcription 

5. Also fix the length of summary.



'''

In [ ]:
'''
1. Getting all the video infomation of our creator and extract transcription for all those videos 

2. Figure out the databse to store transcriptions / embeddings 

3. 






'''